<a href="https://colab.research.google.com/github/rafaeldjsm/estruturas_metalicas/blob/main/Dimensionamento_secao_Ue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dimensionamento de seção U Enrijecida

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from scipy import interpolate

E = 200_000 #MPa
fy = 260 #MPa
fu = 410 #MPa
v = 0.3 #O coeficiente de Poisson do aço, adotado igual a 0,3;
G = 77000 # E/(2*(1+v))

# Seção U enrijecida

## Propriedades Geométricas

$a=b_w-2(r_m+0,5t)$\
$a_m=b_w-t$\
$b=b_f-(r_m+0,5t)$\
$b_m=b_f-t$\
$c=D-(r_m+0,5t)$\
$c_m=D-0,5t$\
$r_m=r_i+0,5t$\
\
$u_1=\frac{\pi}2  r_m$\
\
$u_2=\frac{u_1}2$\
\
$$A=t[a+2(b+c)+4u_1]$$
\
$$x_g=\frac{2t}{A}[b(0,5b+r_m)+(u_1+c)(b+2r_m)]+0,5t$$
\
$$x_0=b_m \left[ \frac {3a_m^2b_m+c_m(6a_m^2-8c_m^2))}{a_m^3+6a_m^2b_m+c_m(8c_m^2-12a_mc_m+6a_m^2)} \right]+x_g-0,5t $$\
$$I_x=2t[0,042a^3+b(0,5a+r_m)^2+2u_1(0,5a+0,637r_m)^2+0,298r_m^3+0,083c^3+0,25(a-c)^2]$$

$$I_y=2t[b(0,5b+r_m)^2+0,083b^3+0,505r_m^3+c(b+2r_m)^2+u_1(b+1,637r_m)^2]-A(x_g-0,5t)^2$$
\
$$C_w=\frac{a_m^2b_m^2t}{12}\left[\frac{2a_m^3b_m+3a_m^2b_m^2+48c_m^4+112b_mc_m^3+48a_mb_mc_m^2+12a_m^2c_m^2+6a_m^3c_m}{6a_m^2b_m+(a_m+2c_m)^3-24a_mc_m^2}\right]$$
 
![alt text](https://github.com/rafaeldjsm/estruturas_metalicas/blob/main/perfilUe.JPG?raw=true "Logo Title Text 1")

In [2]:
#@title ESCOLHA DA SEÇÃO U ENRIJECIDA

list_dim = [[50,25,10,1.2],
[50,25,10,1.5],
[50,25,10,2],
[50,25,10,2.25],
[50,25,10,2.65],
[50,25,10,3],
[75,40,15,1.2],
[75,40,15,1.5],
[75,40,15,2],
[75,40,15,2.25],
[75,40,15,2.65],
[75,40,15,3],
[100,40,17,1.2],
[100,40,17,1.5],
[100,40,17,2],
[100,40,17,2.25],
[100,40,17,2.65],
[100,40,17,3],
[100,40,17,3.35],
[100,50,17,1.2],
[100,50,17,1.5],
[100,50,17,2],
[100,50,17,2.25],
[100,50,17,2.65],
[100,50,17,3],
[100,50,17,3.35],
[125,50,17,2],
[125,50,17,2.25],
[125,50,17,2.65],
[125,50,17,3],
[125,50,17,3.35],
[125,50,17,3.75],
[150,60,20,2],
[150,60,20,2.25],
[150,60,20,2.65],
[150,60,20,3],
[150,60,20,3.35],
[150,60,20,3.75],
[150,60,20,4.25],
[150,60,20,4.75],
[200,75,20,2],
[200,75,20,2.25],
[200,75,25,2.65],
[200,75,25,3],
[200,75,25,3.35],
[200,75,25,3.75],
[200,75,25,4.25],
[200,75,25,4.75],
[200,75,30,6,3],
[200,100,25,2.65],
[200,100,25,3],
[200,100,25,3.35],
[200,100,25,3.75],
[200,100,25,4.25],
[200,100,25,4.75],
[250,85,25,2],
[250,85,25,2.25],
[250,85,25,2.65],
[250,85,25,3],
[250,85,25,3.35],
[250,85,25,3.75],
[250,85,25,4.25],
[250,85,25,4.75],
[250,85,25,6,3],
[250,100,25,2.65],
[250,100,25,3],
[250,100,25,3.35],
[250,100,25,3.75],
[250,100,25,4.25],
[250,100,25,4.75],
[300,85,25,2],
[300,85,25,2.25],
[300,85,25,2.65],
[300,85,25,3],
[300,85,25,3.35],
[300,85,25,3.75],
[300,85,25,4.25],
[300,85,25,4.75],
[300,85,25,6,3],
[300,100,25,2.65],
[300,100,25,3],
[300,100,25,3.35],
[300,100,25,3.75],
[300,100,25,4.25],
[300,100,25,4.75]]


bwlst = []
bflst =[]
Dlst = []
tlst = []

for k in list_dim:
  bwlst.append(k[0])
  bflst.append(k[1])
  Dlst.append(k[2])
  tlst.append(k[3])


df_perfUe = pd.DataFrame([
bwlst,
bflst,
Dlst,
tlst]).T


df_perfUe.index = ['50 x 25 x 10 x 1,2',
'50 x 25 x 10 x 1,5',
'50 x 25 x 10 x 2',
'50 x 25 x 10 x 2,25',
'50 x 25 x 10 x 2,65',
'50 x 25 x 10 x 3',
'75 x 40 x 15 x 1,2',
'75 x 40 x 15 x 1,5',
'75 x 40 x 15 x 2',
'75 x 40 x 15 x 2,25',
'75 x 40 x 15 x 2,65',
'75 x 40 x 15 x 3',
'100 x 40 x 17 x 1,2',
'100 x 40 x 17 x 1,5',
'100 x 40 x 17 x 2',
'100 x 40 x 17 x 2,25',
'100 x 40 x 17 x 2,65',
'100 x 40 x 17 x 3',
'100 x 40 x 17 x 3,35',
'100 x 50 x 17 x 1,2',
'100 x 50 x 17 x 1,5',
'100 x 50 x 17 x 2',
'100 x 50 x 17 x 2,25',
'100 x 50 x 17 x 2,65',
'100 x 50 x 17 x 3',
'100 x 50 x 17 x 3,35',
'125 x 50 x 17 x 2',
'125 x 50 x 17 x 2,25',
'125 x 50 x 17 x 2,65',
'125 x 50 x 17 x 3',
'125 x 50 x 17 x 3,35',
'125 x 50 x 17 x 3,75',
'150 x 60 x 20 x 2',
'150 x 60 x 20 x 2,25',
'150 x 60 x 20 x 2,65',
'150 x 60 x 20 x 3',
'150 x 60 x 20 x 3,35',
'150 x 60 x 20 x 3,75',
'150 x 60 x 20 x 4,25',
'150 x 60 x 20 x 4,75',
'200 x 75 x 20 x 2',
'200 x 75 x 20 x 2,25',
'200 x 75 x 25 x 2,65',
'200 x 75 x 25 x 3',
'200 x 75 x 25 x 3,35',
'200 x 75 x 25 x 3,75',
'200 x 75 x 25 x 4,25',
'200 x 75 x 25 x 4,75',
'200 x 75 x 30 x 6,3',
'200 x 100 x 25 x 2,65',
'200 x 100 x 25 x 3',
'200 x 100 x 25 x 3,35',
'200 x 100 x 25 x 3,75',
'200 x 100 x 25 x 4,25',
'200 x 100 x 25 x 4,75',
'250 x 85 x 25 x 2',
'250 x 85 x 25 x 2,25',
'250 x 85 x 25 x 2,65',
'250 x 85 x 25 x 3',
'250 x 85 x 25 x 3,35',
'250 x 85 x 25 x 3,75',
'250 x 85 x 25 x 4,25',
'250 x 85 x 25 x 4,75',
'250 x 85 x 25 x 6,3',
'250 x 100 x 25 x 2,65',
'250 x 100 x 25 x 3',
'250 x 100 x 25 x 3,35',
'250 x 100 x 25 x 3,75',
'250 x 100 x 25 x 4,25',
'250 x 100 x 25 x 4,75',
'300 x 85 x 25 x 2',
'300 x 85 x 25 x 2,25',
'300 x 85 x 25 x 2,65',
'300 x 85 x 25 x 3',
'300 x 85 x 25 x 3,35',
'300 x 85 x 25 x 3,75',
'300 x 85 x 25 x 4,25',
'300 x 85 x 25 x 4,75',
'300 x 85 x 25 x 6,3',
'300 x 100 x 25 x 2,65',
'300 x 100 x 25 x 3',
'300 x 100 x 25 x 3,35',
'300 x 100 x 25 x 3,75',
'300 x 100 x 25 x 4,25',
'300 x 100 x 25 x 4,75']

style = {'description_width': 'initial'}

Perf_Ue= widgets.Dropdown(
    options=df_perfUe.index,
    value="100 x 40 x 17 x 1,2",
    description='Definição da seção:', 
    style=style,
    disabled=False)

display(Perf_Ue)

Dropdown(description='Definição da seção:', index=12, options=('50 x 25 x 10 x 1,2', '50 x 25 x 10 x 1,5', '50…

In [3]:
# @title Verificando perfil U enrijecido - Dados de entrade a propriedades geométricas

hy, bf, D, tw = df_perfUe.loc[Perf_Ue.value]


#Relação entre os termos da norma e as variáveis usadas
#hy = bw
#tw =t
# D = 17 #mm Tamanho da aba enrijecedora

def prop_geo_Ue(hy, bf, D, tw):
  ri = tw
  rm = ri + 0.5*tw
  u1 = np.pi*rm/2
  u2 = u1/2

  a = hy - 2*(rm + 0.5*tw)
  b = bf - 2*(rm + 0.5*tw)
  c = D - (rm +0.5*tw)
  am = hy - tw
  bm = bf - tw
  cm = D -0.5*tw

  Ag = tw*(a+2*(b+c)+4*u1)

  x_g = (2*tw/Ag)*(b*(0.5*b+rm)+(u1+c)*(b+2*rm))+0.5*tw

  x0 =( (bm*(3*(am**2)*bm + cm*(6*am**2-8*cm**2)) /
        (am**3 + 6*(am**2)*bm + cm*(8*cm**2-12*am*cm+6*am**2)) ) +
        x_g -0.5*tw )

  Ix = 2*tw*( 0.042*a**3 + b*(0.5*a + rm)**2 + 2*u1*(0.5*a + 0.637*rm)**2 +
            0.298*rm**3 + 0.083*c**3 + 0.25*c*(a-c)**2)

  Iy = ( 2*tw*(b*(0.5*b+rm)**2 + 0.083*b**3 + 0.505*rm**3 + c*(b+2*rm)**2 +
            u1*(b+1.637*rm)**2) - Ag*(x_g - 0.5*tw)**2 )

  Cw = ( ( ( (am*bm)**2)*tw/12 )*(2*(am**3)*bm + 3*((am*bm)**2) + 48*(cm**4) +112*bm*(cm**3)+
                          8*am*(cm**3) + 48*am*bm*(cm**2) + 12*((am*cm)**2) +
                          12*(am**2)*bm*cm + 6*(am**3)*cm) /
                          (6*(am**2)*bm + (am+2*cm)**3 - 24*am*cm**2) )/1000000
  Yc = hy/2

  Wx = (Ix/Yc)/1000

  Wy = (Iy/(bf-x_g))/1000

  rx = ((Ix/Ag)**0.5)/10

  ry = ((Iy/Ag)**0.5)/10


  J = ((tw**3)/3)*(a+2*(b+c)+4*u1)/10000

  Ag = Ag/100

  P = Ag*0.785

  Ix = Ix/10000

  Iy = Iy/10000

  x_g = x_g/10
  x0 = x0/10

  r0 = (rx**2 + ry**2 + x0**2)**0.5

  u_pint = (a+2*(b+c)+4*u1 +2*tw)

  h = hy - 2*(tw + ri)
  hy2 = h

  Esb_A = h/tw

  return P, hy, bf, D, tw, h, hy2, Ag, Ix, Wx, rx, Iy, Wy, ry,J, Esb_A,Cw,u_pint,x_g,x0,r0

P, hy, bf, D, tw, h, hy2, Ag, Ix, Wx, rx, Iy, Wy, ry,J, Esb_A,Cw,u_pint,x_g,x0,r0 = prop_geo_Ue(hy, bf, D, tw)

Wxt = Wxc = Wx


columnsUe = (['Massa Linear (Kg/m)', 'd (Altura do perfil mm)', 'bf (mm)','D (mm)',
              'tw Espessura da Alma (mm)','h (Distância entre mesas mm)',
              "d' (Altura Livre da Alma-mm)",
              'Área(cm²)', 'Ix (cm⁴)', 'Wx (cm³)', 'rx (cm)', 'Iy (cm⁴)',
              'Wy (cm³)', 'ry (cm)', 'It (cm⁴)',
              "Esbeltez da Alma (h/tw)", 'Cw (cm⁶)',
              'u (Área de pintura - m²/m)','xg','x0','r0'])

pd.DataFrame([P, hy, bf,D , tw, h, hy2, Ag, Ix, Wx, rx, Iy, Wy, ry,
              J, Esb_A,Cw,u_pint,x_g,x0,r0],columnsUe).T

,Massa Linear (Kg/m),d (Altura do perfil mm),bf (mm),D (mm),tw Espessura da Alma (mm),h (Distância entre mesas mm),d' (Altura Livre da Alma-mm),Área(cm²),Ix (cm⁴),Wx (cm³),rx (cm),Iy (cm⁴),Wy (cm³),ry (cm),It (cm⁴),Esbeltez da Alma (h/tw),Cw (cm⁶),u (Área de pintura - m²/m),xg,x0,r0
0,1.941554,100.0,40.0,17.0,1.2,95.2,95.2,2.473317,38.291366,7.658273,3.93469,6.009975,2.292814,1.558822,0.011872,79.333333,148.492687,208.509734,1.378778,3.36243,5.405335


In [4]:
#@title ## Dados de entrada do comprimento destravado da viga
#@markdown ---
#@markdown ### comprimento destravado em x (cm):
Lx = 770 #@param {type:"number"}

#@markdown ### comprimento destravado em y (cm):
Ly = 257 #@param {type:"number"}

#@markdown ### comprimento destravado de torção(cm):
Lt = 257 #@param {type:"number"}

In [5]:
#@title Resistência ao cisalhamento na maior inércia (9.8.3 Força cortante NBR ABNT_NBR_14762_2010)

kv = 5 # alma semenrijecedores transversais)

if Esb_A <= 1.08*np.sqrt(kv*E/fy):
  Vrd_U = 0.6*fy*hy2*tw/1.1

elif 1.08*np.sqrt(kv*E/fy) < Esb_A < 1.4*np.sqrt(kv*E/fy):
  Vrd_U = 0.65*(tw**2)*((kv*fy*E)**0.5)/1.1
else:
  Vrd_U = 0.905*(tw**3)*((kv*E))/(hy2*1.1)

print("Resistência ao cisalhamento no eixo X = {:.3f}kN".format(Vrd_U/1000))

Resistência ao cisalhamento no eixo X = 13.720kN


In [6]:
#@title Verificação para o início do escoamento da seção efetiva **MSE** (item 9.8.2.1.b)

ene = bf/hy # U enrijecido é Caso b da Tabela 12

if 0.2 <= ene <= 1:
  print("Vefificação do Caso b atendida")

mu2 = D/hy #Relação da aba enrijecedora e altura do perfil

a3 = 81 - 730*ene + 4261*ene**2  -12304*ene**3 + 17919*ene**4 - 12796*ene**5 + 3574*ene**6

#*******************************************
if (0.1 <= mu2 <= 0.2) & (0.2 <= ene <= 1):
  b3 = 0

elif (0.2 < mu2 <= 0.3) & (0.6 < ene <= 1):
  b3 = 0

elif (0.2 < mu2 <= 0.3) & (0.2 <= ene <= 0.6):
  b3 = 320 - 2788*ene+ 13458*ene**2 - 27667*ene**3 + 19167*ene**4
else:
  print("Nenhuma condição atendida")
#*******************************************

kl = a3 - b3 *(mu2 -0.2)

# Momento fletor de flambagem local elástica
Ml = kl*(np.pi**2)*E*(Wxc*1000)/(12*(1-v**2)*((hy/tw)**2)) #N.mm

lamb_p3 = (Wx*fy*1000/Ml)**0.5

if lamb_p3 <= 0.673:
  Wef = Wx
else:
  Wef = Wx*(1-0.22/lamb_p3)/lamb_p3

MRd = Wef*fy/1.1 #N.m

print("O momento fletor resistente da seção para o estado-limite último de início do escoamento da seção"
      "efetiva será de {:.3f} kN.cm".format(MRd/10))

Vefificação do Caso b atendida
O momento fletor resistente da seção para o estado-limite último de início do escoamento da seçãoefetiva será de 181.014 kN.cm


In [7]:
#@title Verificação para flambagem lateral com torção (item 9.8.2.2.b)<br> - <font color='red'>**'Conferir a lançar os esforços solicitantes'**

#@markdown ### Cisalhamento no ponto mais desfavorável (kN ):
Nmx = 1.1 #@param {type:"number"}

#@markdown ### Momento fletor máximo (kNm):
M_mx = 0.55 #@param {type:"number"}

#@markdown ### Momento a 1/4 do eixo medido a partir da extremidade da esquerda.(kNm):
M_a =  0.4125 #@param {type:"number"}

#@markdown ### Momento no meio do vão.(kNm):
M_b = 0.55 #@param {type:"number"}

#@markdown ### Momento a 3/4 do eixo medido a partir da extremidade da esquerda.(kNm):
M_c = 0.4125 #@param {type:"number"}

#@markdown ### Parâmetro de monossimetria:
R_m=1 #@param {type:"number"}

C_b=min(( 12.5*M_mx/(2.5*M_mx+3*M_a+4*M_b+3*M_c) )*R_m,3)

#força axial de flambagem global elástica por flexão em relação ao eixo principal y:
# Ky*Ly Como a viga é biapoiada, adoto Ky =1 

Ney = (np.pi**2)*E*100*Iy/(Ly**2) #N

#força axial de flambagem global elástica por torção:

Nez = (1/(r0**2))*( (np.pi**2)*100*E*Cw/(Lt**2) + 100*G*J) #N

# Ney = 701.7*1000 #Exemplos do exercicio
# Nez = 515.7*1000

# barras com seção duplamente simétrica ou monossimétrica, sujeitas à flexão em torno do eixo de
# simetria (eixo x):
# Me é o momento fletor de flambagem lateral com torção.

Me = C_b*r0*((Ney*Nez)**0.5) #N.cm

lamb0 = (Wxc*fy*100/Me)**0.5

if lamb0 <= 0.6:
  chi_FLT = 1
elif 0.6 < lamb0 < 1.336:
  chi_FLT = 1.11*(1 - 0.278*(lamb0**2))
else:
  chi_FLT = 1/(lamb0**2)

lamb_p4 = (chi_FLT*Wx*fy*1000/Ml)**0.5

if lamb_p4 <= 0.673:
  Wcef = Wxc
else:
  Wcef = Wxc*(1-0.22/lamb_p4)/lamb_p4

MRd_flt = chi_FLT*Wcef*fy/1.1

print("O momento fletor resistente da seção para a Flambagem lateral com torção da seção "
      "efetiva será de {:.3f} kN.cm".format(MRd_flt/10))

O momento fletor resistente da seção para a Flambagem lateral com torção da seção efetiva será de 101.286 kN.cm


In [8]:
#@title Verificação para flambagem distorcional (item 9.8.2.3) - Valores mínimos da relação D/bw de seções

#Para barras com seção U enrijecido e seção Z enrijecido, sob flexão simples em torno do eixo de maior inércia, se
# a relação D/bw for igual ou superior aos valores indicados na Tabela 14, a verificação da flambagem distorcional
# pode ser dispensada.

#df_tab14_NBR14762 = 

df_tab14_NBR = pd.DataFrame({'bf/bw': [round(0.2*(2 + k),1) for k in range(9)],
                  250:   [0.05 for k in range(9)],
                  200:   [0.06 for k in range(9)],
                  125:   [0.1 if k < 2 else 0.09 for k in range(9)],
                  100:   [0.12 if k < 3 else (0.11 if k<5 else 0.1) for k in range(9)],
                  50 :   [0.25 if k < 2 else (0.22 if k<4 else (0.2 if k<7 else 0.19)) for k in range(9)]
                  })

df_tab14_NBR.set_index('bf/bw',inplace=True)

df_tab14_NBR2 = df_tab14_NBR.copy()

df_tab14_NBR.columns = pd.MultiIndex.from_product([ ['bw/t'], [250,200,125,100,50] ])

df_tab14_NBR

bw/t                        
        250   200   125   100   50 
bf/bw                              
0.4    0.05  0.06  0.10  0.12  0.25
0.6    0.05  0.06  0.10  0.12  0.25
0.8    0.05  0.06  0.09  0.12  0.22
1.0    0.05  0.06  0.09  0.11  0.22
1.2    0.05  0.06  0.09  0.11  0.20
1.4    0.05  0.06  0.09  0.10  0.20
1.6    0.05  0.06  0.09  0.10  0.20
1.8    0.05  0.06  0.09  0.10  0.19
2.0    0.05  0.06  0.09  0.10  0.19

In [9]:
#@title FUNÇÃO PARA VERIFICAÇÃO DE RESISTÊNCIA A FLAMBAGEM DISTORCIONAL, INTRPOLAÇÃO LINEAR EM bw/t

bf_bw = bf/hy #bf/bw da norma
bw_t = hy/tw #bf/t da norma
D_bw = D/hy


if bw_t < 50:
  print('Atentar bw/t = {:.2f} é menor que 50'.format(bw_t))
elif bw_t > 250:
  print('Atentar bw/t = {:.2f} é maior que 250'.format(bw_t))

#***************************************************************
def fD_bw(bw_t,bf_bw):
  if (bf_bw in df_tab14_NBR.index)&(bw_t in df_tab14_NBR.columns):
    D_bw_min = df_tab14_NBR.loc[bf_bw,('bw/t',bw_t)]
  elif bw_t == 250:
    D_bw_min = 0.05
  elif bw_t == 200:
    D_bw_min = 0.06

  elif (bw_t == 125)&(bf_bw <= 0.6): # 125
    D_bw_min = 0.1
  elif (bw_t == 125)&(bf_bw >= 0.8):
    D_bw_min = 0.09
  elif (bw_t == 125)&(0.6 < bf_bw < 0.8):
    D_bw_min = -0.05*bf_bw + 0.13

  elif (bw_t == 100)&(bf_bw <= 0.8): #100
    D_bw_min = 0.12
  elif (bw_t == 100)&(0.8 < bf_bw < 1):
    D_bw_min = -0.05*bf_bw + 0.16
  elif (bw_t == 100)&(1 <= bf_bw <= 1.2):
    D_bw_min = 0.11
  elif (bw_t == 100)&(1.2 < bf_bw < 1.4):
    D_bw_min = -0.05*bf_bw + 0.15
  elif (bw_t == 100)&(bf_bw >= 1.4):
    D_bw_min = 0.1


  elif (bw_t == 50)&(bf_bw <= 0.6): #50
    D_bw_min = 0.25
  elif (bw_t == 50)&(0.8 <= bf_bw <= 1):
    D_bw_min = 0.22
  elif (bw_t == 100)&(1 < bf_bw < 1.2):
    D_bw_min = -0.1*bf_bw + 0.3
  elif (bw_t == 50)&(1.2 <= bf_bw <= 1.6):
    D_bw_min = 0.2
  elif (bw_t == 100)&(1.6 < bf_bw < 1.8):
    D_bw_min = -0.05*bf_bw + 0.24
  elif (bw_t == 50)&(1.8 <= bf_bw <= 2):
    D_bw_min = 0.19

  else:
    D_bw_min = -0.001887993421053*bw_t	-0.038404605263158*bf_bw +	0.348260690789474 #Regressão linear com determinação de 97,49% 
  return D_bw_min
#***************************************************************



In [10]:
#@title VERIFICAÇÃO DE RESISTÊNCIA A FLAMBAGEM DISTORCIONAL POR INTRPOLAÇÃO 2D
y = df_tab14_NBR2.index.to_list() #bf/bw
x = df_tab14_NBR2.columns.to_list() #bw/t
z = np.array(df_tab14_NBR2) #D/bw
f = interpolate.interp2d(x, y, z, kind='cubic')

In [11]:
#@title VERIFICAÇÃO DE RESISTÊNCIA A FLAMBAGEM DISTORCIONAL

D_bw_min = float(f(bw_t,bf_bw))

if D_bw < D_bw_min:
  print('Valor de D/bw calculado é inferior ao valor mínimo apresentado pela Tabela 14 da ABNT '
        'NBR 14762:2010, será necessário determinar a resistência à flambagem distorcional da seção.')
else:
  print('Valor de D/bw calculado é superios ao valor mínimo apresentado pela Tabela 14 da ABNT '
        'NBR 14762:2010, não se faz necessário determinar a resistência à flambagem distorcional da seção.')


Valor de D/bw calculado é superios ao valor mínimo apresentado pela Tabela 14 da ABNT NBR 14762:2010, não se faz necessário determinar a resistência à flambagem distorcional da seção.


In [12]:
df_distor_cr = pd.read_csv("https://raw.githubusercontent.com/rafaeldjsm/estruturas_metalicas/main/TabMomDist.csv",decimal=",", index_col='Perfil')

if Perf_Ue.value in df_distor_cr.index:

  Mdist = df_distor_cr.loc[Perf_Ue.value,'Mdist,x (kNm)'] #kN.m

  lamb_dist = (Wx*fy/(Mdist*1000))**0.5

  if lamb_dist < 0.673:
    chi_dist = 1
  else:
    chi_dist = (1 - 0.22/lamb_dist)/lamb_dist

  Mr_dist = chi_dist*Wx*fy/1.1

  print("O momento distorcional reisstente é igual a {:.3f} kNcm".format(Mr_dist/10))
else:
  print("Perfil não listado na tabela de cargas críticas")
  Mr_dist == MRd_flt #Para o caso de não termos momento distorcional tabelado.

MRdx = min(MRd,MRd_flt,Mr_dist) #N.m

print("Momento fletor resistente em x: {:.3f}kN.cm".format(MRdx/10))

lamb_dist,chi_dist, Mr_dist

O momento distorcional reisstente é igual a 181.014 kNcm
Momento fletor resistente em x: 101.286kN.cm


(0.6146767643489407, 1, 1810.13732254786)

#Verificação da combinação de momento fletor e força cortante

Segundo o item 9.8.4 da ABNT NBR 14762:2010, as barras sem enrijecedores transversais na alma e que forem solicitadas pela combinação de momento fletor e força cortante devem satisfazer à seguinte expressão:

$$(\frac{M_{Sd}}{M_{Rd}})^2 + (\frac{V_{Sd}}{V_{Rd}})^2 \leqslant 1$$

In [13]:
#@title VERIFICAÇÃO DOS CRITÉRIOS DE COMBINAÇÃO DE ESFORÇOS SOLICITANTES

N_meio = 0

if (M_mx*1000/MRdx)**2 + (N_meio/Vrd_U/1000)**2 <= 1:
  print('Vefificação da combinação de momento e esforço cortante atendida')

Vefificação da combinação de momento e esforço cortante atendida


In [14]:
#@title VERIFICAÇÃO DA FLECHA

#Momentos Máximos das combinações CT5 (0,8kN.m) e CT7 (5kN.m) pela análise do DMF

Mn = 0.55 #kNm

q = 1.1 #kN.m CT7

#Para a combinação  CT5 e CT6 a flecha máxima é de 1/180
# Considerar combinações raras de serviço, 
#utilizando-se as ações variáveis de mesmo sentido que o da ação permanente.

#Para a combinação  CT7 a flecha máxima é de 1/120
# Considerar apenas as ações variáveis de sentido oposto 
# ao da ação permanente (vento de sucção) com seu valor característico.

lamb_pd = (Mn/(Ml/10000))**0.5

if lamb_pd <= 0.673: #MSE
  Ief = Ix
else:
  Ief = Ix*(1 - 0.22/lamb_pd)

flc = 5*(q/100)*(Lx**4)/(384*(E*100)*Ief)

if flc > Lx/180:
  print("Dimensionamento NÃO atendido quanto ao critério da flecha")
else:
    print("Dimensionamento Atendido quanto ao critério da flecha")

Dimensionamento Atendido quanto ao critério da flecha


In [15]:
#@title CRIAÇÃO DE UM OUTPUT COM OS DADOS DE PERFIS SEGUNDO A NBR 6355 E QUAIS PRECISAM SER ANALISADAS QUANTO A FLAMBAGEM DISTORCIONAL

bfbwlst = []
bwtlst = []
Dbwlst = []
tab_perf = []
for k,j,w,Di in zip(bflst,bwlst,tlst,Dlst):
  bfbwlst.append(k/j)
  bwtlst.append(j/w)
  Dbwlst.append(Di/j)
  tab_perf.append(prop_geo_Ue(j, k, Di, w))

rstDbw1 = [] #Modelo com interpolação em 1d e regressão linear
rstDbw2 = [] #Modelo com interpolação em 2d

for k,(bfbw2,bwt2) in enumerate(zip(bfbwlst, bwtlst)):
  rstDbw1.append(fD_bw(bwt2,bfbw2))
  rstDbw2.append(float(f(bwt2,bfbw2)))


df_perf = pd.DataFrame(tab_perf)
df_perf.columns = columnsUe
df_perf.index = df_perfUe.index
df_perf.index.names = ['Perfis']

df_perf['bf/bw'] = bfbwlst
df_perf['bw/t'] = bwtlst
df_perf['D/bw'] = Dbwlst
df_perf['D/bw_min'] = np.round(rstDbw2,4)

df_perf['Verificação'] = (np.where(df_perf['D/bw'] >= df_perf['D/bw_min'],
                                   'DISPENSA-SE A ANÁLISE DA FLAMBAGEM DISTORCIONAL',
                                   'DEVE-SE VERIFICAR A FLAMBAGEM DISTORCIONA'))

In [16]:
df_perf[df_perf['Verificação'] == 'DISPENSA-SE A ANÁLISE DA FLAMBAGEM DISTORCIONAL']

,Massa Linear (Kg/m),d (Altura do perfil mm),bf (mm),D (mm),tw Espessura da Alma (mm),h (Distância entre mesas mm),d' (Altura Livre da Alma-mm),Área(cm²),Ix (cm⁴),Wx (cm³),rx (cm),Iy (cm⁴),Wy (cm³),ry (cm),It (cm⁴),Esbeltez da Alma (h/tw),Cw (cm⁶),u (Área de pintura - m²/m),xg,x0,r0,bf/bw,bw/t,D/bw,D/bw_min,Verificação
Perfis,,,,,,,,,,,,,,,,,,,,,,,,,,
"100 x 40 x 17 x 1,2",1.941554,100,40,17,1.2,95.2,95.2,2.473317,38.291366,7.658273,3.934690,6.009975,2.292814,1.558822,0.011872,79.333333,148.492687,208.509734,1.378778,3.362430,5.405335,0.40,83.333333,0.17,0.1446,DISPENSA-SE A ANÁLISE DA FLAMBAGEM DISTORCIONAL
"100 x 50 x 17 x 1,2",2.129954,100,50,17,1.2,95.2,95.2,2.713317,44.148230,8.829646,4.033726,10.116428,3.149357,1.930916,0.013024,79.333333,246.608774,228.509734,1.787779,4.281177,6.190950,0.50,83.333333,0.17,0.1418,DISPENSA-SE A ANÁLISE DA FLAMBAGEM DISTORCIONAL
250 x 85 x 25 x 2,7.172538,250,85,25,2.0,242.0,242.0,9.136991,871.507860,69.720629,9.766390,88.981639,14.667003,3.120675,0.121827,121.000000,11477.058675,460.849556,2.433209,6.091600,11.925962,0.34,125.000000,0.10,0.1000,DISPENSA-SE A ANÁLISE DA FLAMBAGEM DISTORCIONAL


In [17]:
df_perf.to_excel('Tabela_perfis_Ue.xlsx')